In [ ]:
from pyzbar import pyzbar
import cv2
from pathlib import Path
from tqdm import tqdm_notebook
import shutil
import pandas as pd
import re

In [ ]:
FOLDER = Path("../data/qr_parsed")

In [ ]:
def find_qr_detection(detections):
    for det in detections:
        if det.type == "QRCODE":
            return det

In [ ]:
GRADE_PATTERN = r"(.+)\s{1}([1-5-]{4})"

In [ ]:
def detect_grades(file_path):
    # some filenames would not be accepted by opencv
    shutil.copy(file_path, "tmp.jpg")
    img = cv2.imread("tmp.jpg")
    
    # perform QR code detection
    detections = pyzbar.decode(img)
    det = find_qr_detection(detections)
    
    if det is None:
        # todo: change to logger
        print("Could not detect", file_name)
        return
      
    qr_data_tokens = det.data.decode("utf-8", "strict").split(";")
      
    for token in qr_data_tokens:
        for match in re.finditer(GRADE_PATTERN, token):
            for grade in match.group(2):
                if grade != "-":
                    course = match.group(1)
            yield {"soubor": Path(file_path).name, "jmeno": qr_data_tokens[1], "predmet": course, "znamka": grade}

In [ ]:
items = []
for file_name in FOLDER.glob("*.*"):  
    for grades in detect_grades(file_name):
        items.append(grades)
    break

df = pd.DataFrame(items)
df

In [ ]:
# store result to excel file
df.to_excel("marks_qr.xls", index=False)

In [ ]:
pd.read_excel("marks_qr.xls")